<a href="https://colab.research.google.com/github/itovvmk/endometriosis/blob/main/Language_and_Health_Pain_in_Corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Updating modules

In [ ]:
# Using Google Drive's files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Installing and updating spacy and language model (Spanish)
!pip install -U spacy

!python -m spacy download es_core_news_lg

2024-01-01 18:00:02.457119: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-01 18:00:02.457198: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-01 18:00:02.469207: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-01 18:00:02.503382: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-01 18:00:08.097038: W tensorflow/compiler/tf2

In [ ]:
# Installing Gensim
!pip install gensim==3.8

In [ ]:
# Importing sci-kit learn and numpy
from sklearn.manifold import TSNE
import numpy as np

In [ ]:
# Importing os
import os

In [ ]:
# Importing other necessary modules
import gensim #solve import errors as in https://stackoverflow.com/questions/69381312/importerror-cannot-import-name-mapping-from-collections-using-python-3-10
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
import pandas as pd

In [ ]:
# Importing other NLP modules
import spacy
nlp = spacy.load('es_core_news_lg')
from nltk.corpus import PlaintextCorpusReader
import nltk
nltk.download('punkt')
nltk.download('rslp')
nltk.download('stopwords')
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Stopwords
stops_nltk = nltk.corpus.stopwords.words('spanish')
novas_stops = ['fondo', 'dávila','e', 'cachai', 'cachay', 'poh','eh', 'po', 'ah', 'pa', 'así', 'ay'
               'em','m','xxx', 'inaud', 'aah', 'risas','súper','entretenido', 'venezuela', 'antofagasta', 'er', 'él','yo',
               'entonces', 'si', 'sí', 'pucha', 'puta',
               'quizás', 'ser', 'sé', 'igual','cha', 'pu', 'allí',
               'ahí', 'claro', 'va', 'bueno','forma','alguna','doctorado',
               'ejemplo', 'hecho','quizá', 'tal', 'realidad', 'entrecomillas', 'súper',
               'comillas', 'indisa', 'antropología', 'antropologa','sociales','chiloé','acá'
               'tema', 'r', 'l','s','h','bonito', 'años', 'año','cosa','cosas','d','copita']
stops_nltk.extend(novas_stops)
stops_spacy = nlp.Defaults.stop_words
stops = set(stops_nltk) | stops_spacy

In [ ]:
# Defining folder in Google Drive
from nltk.corpus import PlaintextCorpusReader

corpus_root = '/content/drive/My Drive/RiCL/entrevistas'
wordlists = PlaintextCorpusReader(corpus_root, '.*')

# Creating collection list
entrevistas = wordlists.fileids()

corpus = list()

In [ ]:
""" Defining function to tokenize and lemmatize strings
"""
def preproc(string):
    tokens = nltk.tokenize.word_tokenize(string)
    tokens = [t.lower() for t in tokens if t.isalpha()]
    tokens = [i for i in tokens if i not in stops]
    return tokens

def lema(string):
    doc = nlp(string)

    return [token.lemma_ for token in doc]

In [ ]:
# Transforming corpus in list of strings
corpus = []
for entrevista in entrevistas:
    strings = wordlists.raw(entrevista)
    corpus.append(strings)

# Removing interviewers' speeches
regex = r"M*:.*?E:"

colecao_entrevistadas = []
for i in corpus:
    achar_regex = re.findall(regex, i, re.DOTALL)
    entrevistadas = ' '.join(achar_regex)
    colecao_entrevistadas.append(entrevistadas)

In [ ]:
len(colecao_entrevistadas)

30

In [ ]:
# Cleaning, tokenizing, and lemmatizing
tokens = [preproc(i) for i in colecao_entrevistadas]
tokens_strings = [' '.join(i) for i in tokens]
lemas = [lema(i) for i in tokens_strings]

# Word2Vec implementation

In [ ]:
# WORD2VEC for corpus in Spanish
from gensim.models import Word2Vec
# size: dimension of vectors
# window: highest distance (in terms) from the target word
# min_count: minimum frequency count of the chosen terms
model = Word2Vec(lemas, size=120, window=10, min_count=10)

# total_examples: number of sentences for training
model.train(lemas, total_examples=30, epochs=100)

(1971155, 3632100)

# Most similar terms

# Definitions of menstruation

In [ ]:
# termos similares
model.most_similar('menstruación', topn=10)

<ipython-input-278-9ebf28e0e23b>:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('menstruación', topn=10)


[('abundante', 0.6191399693489075),
 ('hueso', 0.5212849378585815),
 ('hemorragia', 0.46438682079315186),
 ('doloroso', 0.4471602439880371),
 ('cortar', 0.42838358879089355),
 ('anticonceptivo', 0.41756123304367065),
 ('tremendo', 0.4049299359321594),
 ('durar', 0.3983049690723419),
 ('severo', 0.3864632546901703),
 ('sangrar', 0.3787758946418762)]

In [ ]:
# termos similares
model.most_similar('regla', topn=10)

<ipython-input-279-22afe12de5db>:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('regla', topn=10)


[('abundante', 0.49064308404922485),
 ('empezar', 0.43338245153427124),
 ('desmayar', 0.39458146691322327),
 ('mes', 0.38473016023635864),
 ('doloroso', 0.37647801637649536),
 ('oportunidad', 0.37431246042251587),
 ('sabi', 0.3717849850654602),
 ('invalidante', 0.3545975983142853),
 ('durar', 0.3523443043231964),
 ('típico', 0.35159099102020264)]

# Definitions of endometriosis

In [ ]:
model.most_similar('endometriosis', topn=10)

<ipython-input-281-89715920f301>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('endometriosis', topn=10)


[('especialista', 0.5998467206954956),
 ('investigar', 0.4279727637767792),
 ('profundo', 0.4089559316635132),
 ('sospechar', 0.3975067734718323),
 ('enterar', 0.3711698651313782),
 ('resonancia', 0.369948148727417),
 ('común', 0.3680271506309509),
 ('hacer él', 0.35261496901512146),
 ('grado', 0.3495524525642395),
 ('información', 0.33816033601760864)]

# Definitions of pain

In [ ]:
model.most_similar('dolor', topn=10)

<ipython-input-282-0a8877908de0>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('dolor', topn=10)


[('constante', 0.5843042731285095),
 ('contracción', 0.5290500521659851),
 ('espalda', 0.4617069363594055),
 ('abdomen', 0.45693260431289673),
 ('intenso', 0.4491477906703949),
 ('hueso', 0.43398958444595337),
 ('analgésico', 0.43102115392684937),
 ('abundante', 0.42961353063583374),
 ('niña', 0.4243719279766083),
 ('molestia', 0.41971537470817566)]

In [ ]:
model.most_similar('doler', topn=10)

<ipython-input-283-d14c8c2ecf25>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('doler', topn=10)


[('reglar', 0.4419650435447693),
 ('durar', 0.42462119460105896),
 ('normal', 0.40757301449775696),
 ('espalda', 0.40254896879196167),
 ('gana', 0.3937356472015381),
 ('dolor', 0.35173511505126953),
 ('descansar', 0.3349992632865906),
 ('regla', 0.33118951320648193),
 ('olvidar', 0.31992366909980774),
 ('guata', 0.3161216974258423)]